ravdess filenames identifiers --
Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
Vocal channel (01 = speech, 02 = song).
Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the 'neutral' emotion.
Statement (01 = "Kids are talking by the door", 02 = "Dogs are sitting by the door").
Repetition (01 = 1st repetition, 02 = 2nd repetition).
Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

In [ ]:
import os
from google.colab import drive
import cv2
import pickle
import dlib
import sys
import numpy as np
import matplotlib.pyplot as plt
!pip install -q --upgrade imutils
!pip install -q --upgrade dlib
!pip install -q deepface
from deepface import DeepFace
from deepface.commons import functions
import imutils
!pip install -q face-alignment
import face_alignment
from imutils import face_utils

from IPython.display import Image
from google.colab.patches import cv2_imshow
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

drive.mount('/content/drive')

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# choose where you want your project files to be saved
project_folder = "FERcapstone/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

In [ ]:

def VideoToFrame(videosfolder, framesfolder):
    

    cwd = os.getcwd()

    videosd = os.path.join(cwd, videosfolder)
    framesd = os.path.join(cwd, framesfolder)

    # reading the vodeos in the directory
    videolist = os.listdir(videosd)

    # making the vids to frames 
    for i in range(0, len(videolist)):

        print("Progress: {}/{}".format(i+1, len(videolist)))
        vtitle = os.path.join(*(videosd, videolist[i]))
        emotion = int(videolist[i].split("-")[2]) - 1

        # makingthe directory for the frames
        framedir = os.path.join(*(framesd, str(i)))
        if not os.path.exists(framedir):
            os.makedirs(framedir)

        vidcap = cv2.VideoCapture(vtitle)
        success,image = vidcap.read()
        count = 0
        success = True
        while success:
            idx = 'frame_' + str(count) + '.png'
            frame_n = os.path.join(*(framedir, idx))

            cv2.imwrite(frame_n, image)
            success,image = vidcap.read()
            count += 1


        #recording the emotion annotations 
        emotionpickle = {"emotion": emotion}
        with open(framedir + "/emotion.pickle", 'wb') as f:
            pickle.dump(emotionpickle, f, protocol = 2)

    #to control when conversion is ran
def runconvert2frames():
    VideoToFrame('dataset/TestVideos/', 'dataset/TestFrames/')
    VideoToFrame('dataset/TrainingVideos/', 'dataset/TrainingFrames/')

           

In [ ]:
#this section to initialize the backend variables and the model used for training

project_directory = os.getcwd()
print("Project directory: ", project_directory)


startindex=0
skip=0
maxcount=1000
framescount = 30
# emption classes from ravdess 
numberemotions = 8
emotions = np.zeros(numberemotions)

# number of instances to get from the folder
maxfolderinstances = 1000

# frames being saved will ahve this resolutioon - 
frameresolution = [300, 300, 1]

#start with this index when getting instances from the directory
startindex = 0

# number of instances to get from the directory 
maxarraycount = 1000

# saving the pickled data every 500 processed folders
savingindex = 500

# face detector treshold
detectorthreshold = 0.8


'''
modelFile = os.path.join(project_directory, "models/opencv_face_detector_uint8.pb")
configFile = os.path.join(project_directory, "models/opencv_face_detector.pbtxt")
face_detector = cv2.dnn.readNetFromTensorflow(modelFile, configFile)
'''

modelFile = os.path.join(project_directory, "models/res10_300x300_ssd_iter_140000.caffemodel")
configFile = os.path.join(project_directory, "models/deploy.prototxt.txt")
face_detector = cv2.dnn.readNetFromCaffe(configFile, modelFile)

# ---- trying a different face detector from dlib to see if the issue is resolved

detector = dlib.get_frontal_face_detector()

# 
predictor = dlib.shape_predictor(os.path.join(project_directory, "models/shape_predictor_68_face_landmarks.dat"))

# ---- also changing the face aligner to check is issue is resolved
#fa = FaceAligner(predictor, desiredFaceWidth=int(frames_resolution[0]), desiredFaceHeight=int(frames_resolution[1]))
#fa = FaceAligner(predictor, desiredFaceWidth = 64)
fa = FaceAligner(predictor, desiredFaceWidth = 256)
#fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)


# making the folders 
trainingframesfolder = os.path.join(project_directory, "dataset/TrainingFrames")
testframesfolder = os.path.join(project_directory, "dataset/TestFrames")

#using pickle to dave the data liek the tutorial 
trainingsavefolder = os.path.join(project_directory, "dataset/resolution_{}x{}x{}_train.pickle".format(frameresolution[0], frameresolution[1], frameresolution[2]))
testsavefolder = os.path.join(project_directory, "dataset/resolution_{}x{}x{}_test.pickle".format(frameresolution[0], frameresolution[1], frameresolution[2]))


In [ ]:
def onehotencode(number, numemotions):
    arraynp = np.zeros((1, numemotions), dtype=int)
    arraynp[0][number] = 1
    return arraynp[0]

def getfolderdata(framesfolder, savefolder):
    data = []

    #sorting the frame folders in order from directory
    framesfolders = sorted(os.listdir(framesfolder), key=int)

    for i, foldername in enumerate(framesfolders):
        
        if i < startindex:
            continue

        if(i%savingindex==0 and i!=0):
            with open(os.path.join(project_directory, savefolder), 'wb') as handle:
                pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

        folderpath = os.path.join(*(framesfolder, foldername))

        # load emotion number for the annotation file in the folder
        with open(os.path.join(*(folderpath, "annotation.pickle")), 'rb') as handle:
            annotation = pickle.load(handle)

        #getting the correct emotion pickle data
        emotion = annotation["emotion"]

        # if directory exist and contain enough frames for 1 array
        if (os.path.exists(folderpath)) and (len(os.listdir(folderpath)) - 1 >= framescount):

            if(emotions[emotion] < maxcount):

                # loop through all frames
                length = len(os.listdir(folderpath)) - 1
                framearrcount = 0
                index = startindex
                images = []
                while (index < length) and (maxfolderinstances > framearrcount) :

                    image_path = os.path.join(folderpath, "frame_{}.png".format(index))
                    image = cv2.imread(image_path, 1)

                    print(index)
                    #(h, w) = image.shape[:2]

                    # --- ALIGNINMENT AND FACE DETECTION DONE HERE USING DEEPFACE -- RETURNS A NUMPYARRAY --- FACE FOUND HERE AND ALIGNED
                    faceobjects = DeepFace.extract_faces(img_path = image, target_size = (300, 300), enforce_detection=False)

                    for faceobject in faceobjects:
                      img = faceobject["face"]
                      detectedface = img.copy()

                      if detectedface.size != 0:

                         # resize and normalization done at this step 
                        if(frameresolution[2] == 1):

                          detectedface = cv2.cvtColor(detectedface, cv2.COLOR_BGR2GRAY)
                          # detectedface = cv2.resize(detectedface, (frameresolution[0], framesresolution[1]))
                          detectedface = cv2.normalize(detectedface, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                          images.append(detectedface)
                          #plt.imshow(detectedface)
                          #plt.show()

                          # skip 'skip' next frames
                          index += skip

                    index += 1


                    # if True, saves array of 'framescount' images
                    if len(images) == framescount:
                        item = {"emotion": np.asarray(onehotencode(emotion, numberemotions)), "images": np.asarray(images)}
                        images = []
                        framearrcount += 1
                        emotions[emotion] += 1
                        data.append(item)

                    if (framearrcount >= maxarraycount):
                        break

                print("--progress-- {}/{} \t emotion added: \t emotion: {} \t{} arrays of {} images".format(i, len(framesfolders), emotion, framearrcount, framescount))

            else:
                print("--progress-- {}/{} \t emotion added: {} \t emotion data limit reached".format(i, len(framesfolders), emotion))

        else:
            print(" Error with the directory - make sure directory exsists")

    return data




# add flipped emotion -- adds more data for the chosen emotion index -- follows ravdess emotion index
def flipemotiondata(data, emotionindex):
    for item in list(data):
        eindex = np.where(item["emotion"] == 1)[0][0]
        if eindex == emotionindex:
            flippedarray = []
            for image in item["images"]:
                flippedarray.append(cv2.flip(image, 0))
            flippedarray = np.asarray(flippedarray)
            newflipped = {"emotion": item["emotion"], "images": flippedarray}
            data.append(newflipped)



# to measure how the emotion data is distributed among the emotions 
def datadistibution(data):
    emotiondata = np.array([x["emotion"] for x in data])
    emotions = np.zeros(numberemotions)
    for i, x in enumerate(emotiondata):
        index = np.where(x == 1)[0][0]
        emotions[index] += 1

    return emotions


# need to call before saving the data depending on the maximum count allowed 
def predatasave(data, maxcount):
    shuffdata = np.asarray(data)
    np.random.shuffle(shuffdata)
    #making the emotion
    resultdata = []
    emotions = np.zeros(numberemotions)
    emotiondata = np.array([x["emotion"] for x in shuffdata])

    for j in range(0, len(shuffdata)):
        index = np.where(emotiondata[j] == 1)[0][0]

        if emotions[index] < maxcount:
            emotion = np.zeros(numberemotions)
            emotion[index] = 1
            sample = shuffdata[j]
            sample["emotion"] = emotion
            resultdata.append(sample)
            emotions[index] += 1

    print("d]distribution of the data depending on emotion: ", emotions)
    return resultdata



#if presavedata works, call this to saave the input data to a pickle file - follow tutorial
def datasave(data, filename):
    filename = os.path.join(project_directory,"dataset/{}".format(filename))
    with open(filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#call the methods sequentially to track errors 

testingdata = getfolderdata(testframesfolder, testsavefolder)
trainingdata = getfolderdata(trainingframesfolder, trainingsavefolder)

print("data distribution -")

#looking at the emotions distribution for the test and training data based on emotions
print("train data -")
print(datadistibution(trainingdata))
print("test data -")
print(datadistibution(testingdata))


#to even out the emotionsdata for the eneutral emotion -- emotion index 0
print("Neutral emotion data getting flipped..")
flipemotiondata(trainingdata, 0)
flipemotiondata(testingdata, 0)

#looking at the emotion distribution after adding more Neutral emotion data 
print("data distribution -- post data flip")
print("train data -")
print(datadistibution(trainingdata))
print("test data -")
print(datadistibution(testingdata))


#to control the max amount of data read -- THIS EFFECTS THE PICKLE DATA SAVE
numbertraindata = 500
numbertestdata = 24

# form
test_data_4_save = predatasave(testingdata, numbertestdata)
train_data_4_save = predatasave(trainingdata, numbertraindata)

In [ ]:

# performing save in a deperate block
datasave(test_data_4_save, "resolution_{}x{}x{}__{}__TESTdata.pickle".format(frameresolution[0], frameresolution[1], frameresolution[2], numbertestdata))
datasave(train_data_4_save, "resolution_{}x{}x{}__{}__TRAINdata.pickle".format(frameresolution[0], frameresolution[1], frameresolution[2], numbertraindata))

In [ ]:
#once the pickle data is paclaged and save din the correct directory - using the pickled data ot train the model

from keras.models import load_model
import numpy as np
import os
import pickle
import keras

#to access the pickle files
folderpath = os.getcwd()
print(" current project directory: ", folderpath)

#double check the name of the picklek files -- they change as the parameters of the data count change 
train_path = "dataset/resolution_300x300x1__500__TRAINdata.pickle"
test_path = "dataset/resolution_300x300x1__24__TESTdata.pickle"
emotions_count = 8



#following pickle file model training tutorial for this part
def splitemotionimage(data):
    image = []
    emotion = []
    for item in data:
        for i in range(0, len(item["images"]), 1):
            image.append(np.expand_dims(item["images"][i],axis=3))
            emotion.append(item["emotion"])
    return np.asarray(image), np.asarray(emotion)


#loading pickle data  

with open(os.path.join(folderpath, test_path), 'rb') as handle:
        testingpickledata = pickle.load(handle)
with open(os.path.join(folderpath, train_path), 'rb') as handle:
        trainingpickledata = pickle.load(handle)

#i-image, f-frame -- splitting the test and train pickle data 
iTEST, fTEST = splitemotionimage(testingpickledata)
iTRAIN, fTRAIN = splitemotionimage(trainingpickledata)


#making model and training params 
modelpath = "models/BaseEmotionClassifyModel.hdf5"
model = load_model(modelpath, compile=False)
learnrate = 0.005
numepochs = 10
batchsize = 8
decayrate = 0.01
momentrate = 0.9 

#choosing sgd jsut for the initial testing 
sgd = keras.optimizers.SGD(lr=learnrate, momentum=momentrate, nesterov=True, decay=decayrate)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


model.fit(iTRAIN,fTRAIN, epochs=numepochs, validation_data=(iTEST, fTEST), batch_size=batchsize)

#saving the new model to be used for testing
#changes the version that is saved everytime it it run

anotherindex = 0
newname = "models/EmotionClassifyVersion_{}.hdf5".format(anotherindex)
newname = os.path.join(folderpath, newname)

model.save(newname)
print(" new model saved as {}".format(newname))
anotherindex = anotherindex+1
       